# 파이썬스러운 객체

파이썬 데이터 모델 덕분에 사용자가 정의한 자료형도 내장 자료형과 마찬가지로 자연스럽게 동작할 수 있다.
상속하지 않고도 덕 타이핑 배커니즘을 통해 이 모든 것이 가능하다.
단지 객체에 필요한 메서드를 구현하면 기대한 대로 동작한다.

## @classmethod와 @staticmethod

`@classmethod`는 메서드가 호출되는 방식을 변경해서 클래스 자체를 첫 번째 인수로 받게 만들며 `frombytes()`같은 대안 생성자를 구현하기 위해 주로 사용된다.

반대로 `@staticmethod`데커레이터는 메서드가 특별한 첫 번째 인수를 받지 않도록 메서드를 변경한다.
본질적으로 정적 메서드는 모듈 대신 클래스 본체 안에 정의된 평범한 함수일 뿐이다.

In [1]:
# classmethod와 staticmethod의 동작 비교

class Demo:
  @classmethod
  def klassmeth(*args):
    return args
  @staticmethod
  def statmeth(*args):
    return args

print(Demo.klassmeth())
print(Demo.klassmeth('spam'))
print(Demo.statmeth())
print(Demo.statmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


## 포맷된 출력
`format()` 내장 함수와 `str.format()` 메서드는 실제 포맷 작업을 `__format__(format_spec)` 메서드에 위임한다.
format_spec은 포맷 명시자로서, 다음 두 가지 방법 중 하나를 통해 지정한다.
- format(my_obj, format_spec)의 두 번째 인수
- str.format()에 사용된 포맷 문자열 안에 {}로 구분한 대체 필드 안에서 콜론 뒤의 문자열

## 해시 가능한 Vector2d

Vector2d를 해시 가능하게 만들려면 `__hash__()`메서드를 구현해야 한다.
해시 가능하단 말의 의미는 Vector2d 객체를 불변형으로 만들어야 한다.


In [18]:
class Vector2d:
  typecode = 'd'

  def __init__(self, x, y):
    self.__x = float(x)
    self.__y = float(y)

  @property
  def x(self):
    return self.__x

  @property # 프로퍼티의 게터 메서드를 나타낸다.
  def y(self):
    return self.__y

  def __iter__(self):
    return (i for i in (self.x, self.y))

  def __hash__(self):
    return hash(self.x) ^ hash(self.y)

v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)

print(hash(v1), hash(v2))
set([v1, v2])

7 384307168202284039


{<__main__.Vector2d at 0x10ff01220>, <__main__.Vector2d at 0x10ff01970>}

## 파이썬 에서의 비공개 속성과 보호된 속성
파이썬에서는 `private` 수정자가 있는 자바와 달리 비공개 변수를 생성할 수 있는 방법은 없지만, 서브클래스에서 '비공개' 성격의 속성을 실수로 변경하지 못하게 하는 간단한 메커니즘은 있다.


In [24]:
# name mangling

v1 = Vector2d(3, 4)
print(v1.__dict__)
v1._Vector2d__x

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}


3.0

이름 장식(naming mangling)은 안전을 제공하지만, 보안 기능은 아니다. 실수로 접근하는 것을 막도록 설계되어있지만 고의적인 악용을 막지는 못한다.

파이썬 문서의 일부에서는 단일 언어바로 시작하는 속성을 protected 속성이라고 부르기도 한다.
속성명 앞에 언더바 하나를 붙이더라도 파이썬 인터프리터가 별도로 특별히 처리하는 것은 없지만, 클래스 외부에서 그런 속성에 접근하지 않는 것은 파이썬 프로그래머 사이에 일종의 금기처럼 자리 잡혀 있다.

## __slots__ 클래스 속성으로 공간 절약하기
기본적으로 파이썬은 객체 속성을 각 객체 안의 `__dict__`라는 딕셔너리형 속성에 저장한다.
딕셔너리는 빠른 접근 속도를 제공하기 위해 내부에 해시 테이블을 유지하므로 메모리 사용량 부담이 상당히 크다.
만약 속성이 몇개 없는 수백만 개의 객체를 다룬다면, `__slots__` 클래스 속성을 이용해서 메모리 사용량을 엄청나게 줄일 수 있다.
`__slots__` 속성은 파이썬 인터프리터가 객체 속성을 딕셔너리 대신 튜플에 저장하게 만든다.

In [ ]:
class Vector2d:
  typecode = 'd'

  def __init__(self, x, y):
    self.__x = float(x)
    self.__y = float(y)

  @property
  def x(self):
    return self.__x

  @property # 프로퍼티의 게터 메서드를 나타낸다.
  def y(self):
    return self.__y

  def __iter__(self):
    return (i for i in (self.x, self.y))

  def __hash__(self):
    return hash(self.x) ^ hash(self.y)

  def __slots__ = ('__x', '__y')

수백만 개의 숫자 데이터를 처리하는 경우에는 NumPy를 사용하는 것이 좋다. NumPy는 메모리를 효율적으로 사용할 뿐만 아니라 숫자 처리에 상당히 최적화된 함수를 가지고 있으며, 그중 배열 전체를 한꺼번에 처리하는 함수도 많이 있다.
